In [1]:
import numpy as np 
import pandas as pd

C:\Users\singhy\AppData\Local\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\singhy\AppData\Local\Temp\ipykernel_5776\118197663.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data_dir = "C:/Users/singhy/Desktop/Chicago/cps_data/inflation/raw_data"
output_dir = "C:/Users/singhy/Desktop/Chicago/cps_data/inflation/output"

df = pd.read_csv(f"{data_dir}/adp/ADP_PAY_history.csv")
cpi = pd.read_csv(f"{data_dir}/CPI/CPIAUCSL.csv")


In [56]:
temp = df[df['category'] == 'Job Stayer']
keep = ['date', 'median pay change']
stayer = temp[keep]

In [57]:
stayer = stayer.rename(columns={'median pay change':'delta_w_stay'})

In [58]:
temp = df[df['category'] == 'Job Changer']
keep = ['date', 'median pay change']
switcher = temp[keep]
switcher = switcher.rename(columns={'median pay change':'delta_w_switch'})

In [59]:
data = switcher.merge(stayer, on=['date'])

In [60]:
data['date'] = pd.to_datetime(data['date'])

In [61]:
data['diff'] = data['delta_w_switch'] - data['delta_w_stay']

In [62]:

cpi = cpi.iloc[10:].reset_index(drop=True)

cpi = cpi.rename(columns={
                            'observation_date': 'date', 
                            'CPIAUCSL':               'P'
})

cpi['date'] = pd.to_datetime(cpi['date'])
cpi['P'] = pd.to_numeric(cpi['P'], errors='coerce')
cpi['P_12m_change'] = cpi['P'].pct_change(periods=12) * 100

In [63]:
data = data.merge(cpi, on =['date'])

In [64]:
data.to_csv(f"{output_dir}/data/adp_wage.csv", index = False )